# Testing cosmogan
Aug 25, 2020

Borrowing pieces of code from : 

- https://github.com/pytorch/tutorials/blob/11569e0db3599ac214b03e01956c2971b02c64ce/beginner_source/dcgan_faces_tutorial.py
- https://github.com/exalearn/epiCorvid/tree/master/cGAN

In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml

In [2]:
%matplotlib widget

In [3]:
def f_load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    return config


In [4]:
ls

1_main_code/        cosmogan_train.ipynb         launch_train_pytorch.ipynb
2_analysis/         generate_images.ipynb
batch_analysis.sh*  launch_compute_chisqr.ipynb


In [6]:
config_file='1_main_code/config_128.yaml'
config_dict=f_load_config(config_file)
print(config_dict)

{'description': 'GAN', 'data': {'ip_fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'op_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'}, 'training': {'workers': 2, 'nc': 1, 'nz': 64, 'ngf': 64, 'ndf': 64, 'lr': 0.0002, 'beta1': 0.5, 'kernel_size': 5, 'stride': 2, 'g_padding': 2, 'd_padding': 2, 'image_size': 128, 'flip_prob': 0.01}}


In [7]:

workers=config_dict['training']['workers']
nc=config_dict['training']['nc']
nc,nz,ngf,ndf=config_dict['training']['nc'],config_dict['training']['nz'],config_dict['training']['ngf'],config_dict['training']['ndf']
lr,beta1=config_dict['training']['lr'],config_dict['training']['beta1']
kernel_size,stride=config_dict['training']['kernel_size'],config_dict['training']['stride']
g_padding,d_padding=config_dict['training']['g_padding'],config_dict['training']['d_padding']
image_size=config_dict['training']['image_size']
ip_fname=config_dict['data']['ip_fname']
op_loc=config_dict['data']['op_loc']

ngpu=1
batch_size=128

In [8]:
manualSeed=2120
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)

Random Seed:  2120
cpu


In [9]:
# # Set random seed for reproducibility
# manualSeed = 999
# #manualSeed = random.randint(1, 10000) # use if you want new results
# print("Random Seed: ", manualSeed)
# random.seed(manualSeed)
# torch.manual_seed(manualSeed)

# # Root directory for dataset
# dataroot = "data/celeba"
# # Number of workers for dataloader
# workers = 2
# # Batch size during training
# batch_size = 50
# # Spatial size of training images. All images will be resized to this
# #   size using a transformer.
# image_size = 128

# nc = 1 # Number of channels in the training images. For color images this is 3
# nz = 64 # Size of z latent vector (i.e. size of generator input)
# ngf = 64 # Size of feature maps in generator
# ndf = 64 # Size of feature maps in discriminator
# num_epochs = 40 # Number of training epochs

# lr = 0.0002 # Learning rate for optimizers
# beta1 = 0.5 # Beta1 hyperparam for Adam optimizers
# ngpu = 1 # Number of GPUs available. Use 0 for CPU mode.

# device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# print(device)

# kernel_size,stride=5,2
# g_padding,d_padding=2,2

In [10]:
# ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname)[:10000].transpose(0,1,2,3)
t_img=torch.from_numpy(img)
img.shape,t_img.shape

((10000, 1, 128, 128), torch.Size([10000, 1, 128, 128]))

In [11]:
dataset=TensorDataset(torch.Tensor(img))
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=1,drop_last=True)

len(dataset),(dataset[0][0]).shape

(10000, torch.Size([1, 128, 128]))

In [12]:
img.shape,t_img.shape

((10000, 1, 128, 128), torch.Size([10000, 1, 128, 128]))

In [13]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)



In [14]:
# Generator Code
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # nn.ConvTranspose2d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
            nn.Linear(nz,nc*ngf*8*8*8),# 32768
            nn.BatchNorm2d(nc),
            nn.ReLU(True),
            View(shape=[-1,ngf*8,8,8]),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size, stride, g_padding, output_padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, kernel_size, stride, g_padding, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, kernel_size, stride, g_padding, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, kernel_size, stride,g_padding, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
    
    def forward(self, input):
        return self.main(input)


# Discriminator Code
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            # nn.Conv2d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
            nn.Conv2d(nc, ndf,kernel_size, stride, d_padding,  bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Flatten(),
            nn.Linear(nc*ndf*8*8*8, 1)
        )

    def forward(self, input):
        return self.main(input)



In [15]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)


# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)


Generator(
  (main): Sequential(
    (0): Linear(in_features=64, out_features=32768, bias=True)
    (1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): View()
    (4): ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): ConvTranspose2d(64, 1, kernel_size=(5, 5), s

In [16]:
summary(netG,(1,1,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 1, 1, 32768]       2,129,920
       BatchNorm2d-2          [-1, 1, 1, 32768]               2
              ReLU-3          [-1, 1, 1, 32768]               0
              View-4            [-1, 512, 8, 8]               0
   ConvTranspose2d-5          [-1, 256, 16, 16]       3,276,800
       BatchNorm2d-6          [-1, 256, 16, 16]             512
              ReLU-7          [-1, 256, 16, 16]               0
   ConvTranspose2d-8          [-1, 128, 32, 32]         819,200
       BatchNorm2d-9          [-1, 128, 32, 32]             256
             ReLU-10          [-1, 128, 32, 32]               0
  ConvTranspose2d-11           [-1, 64, 64, 64]         204,800
      BatchNorm2d-12           [-1, 64, 64, 64]             128
             ReLU-13           [-1, 64, 64, 64]               0
  ConvTranspose2d-14          [-1, 1, 1

In [17]:
summary(netD,(1,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,600
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         LeakyReLU-3           [-1, 64, 64, 64]               0
            Conv2d-4          [-1, 128, 32, 32]         204,800
       BatchNorm2d-5          [-1, 128, 32, 32]             256
         LeakyReLU-6          [-1, 128, 32, 32]               0
            Conv2d-7          [-1, 256, 16, 16]         819,200
       BatchNorm2d-8          [-1, 256, 16, 16]             512
         LeakyReLU-9          [-1, 256, 16, 16]               0
           Conv2d-10            [-1, 512, 8, 8]       3,276,800
      BatchNorm2d-11            [-1, 512, 8, 8]           1,024
        LeakyReLU-12            [-1, 512, 8, 8]               0
          Flatten-13                [-1, 32768]               0
           Linear-14                   

In [18]:
# def f_size(ip):
#     p=2;s=2
# #     return (ip + 2 * 0 - 1 * (p-1) -1 )/ s + 1

#     return (ip-1)*s - 2 * p + 1 *(5-1)+ 1 + 1

# f_size(128)

In [19]:
# Initialize BCELoss function
# criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size, 1, 1, nz, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


In [20]:
### Spectrum code

In [21]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

def f_compute_spectrum(arr):
    y1=np.fft.fft2(arr)
    y2=abs(y1)
    z1=f_radial_profile(y2)
    return(z1)
    
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk


### Code ###
def f_image_spectrum(x,num_channels):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    print(x.shape)
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        sdev[i]=np.std(batch_pk,axis=0)
    mean=np.array(mean)
    sdev=np.array(sdev)
    return mean,sdev



In [22]:
def f_torch_radial_profile(img, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    
    y,x=torch.meshgrid(torch.arange(0,img.shape[0]),torch.arange(0,img.shape[1])) # Get a grid of x and y values
    if center[0]==None and center[1]==None:
        center = torch.Tensor([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers

    # get radial values of every pair of points
    r = torch.sqrt((x - center[0])**2 + (y - center[1])**2)
    r= r.int()

#     print(r.shape,img.shape)
    # Compute histogram of r values
    tbin=torch.bincount(torch.reshape(r,(-1,)),weights=torch.reshape(img,(-1,)).type(torch.DoubleTensor))
    nr = torch.bincount(torch.reshape(r,(-1,)))
    radialprofile = tbin / nr
    
    return radialprofile


def f_torch_compute_spectrum(arr):
    y1=torch.rfft(arr,signal_ndim=2,onesided=False)
    ## Absolute value of each complex number (last index is real/imag part)
    y2=torch.sqrt(y1[:,:,0]**2+y1[:,:,1]**2)
    z1=f_torch_radial_profile(y2)
    return(z1)


def f_torch_compute_batch_spectrum(arr):
    
    batch_pk=torch.stack([f_torch_compute_spectrum(i) for i in arr])
    
    return batch_pk


### Code ###
def f_torch_image_spectrum(x,num_channels):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_torch_compute_batch_spectrum(arr)
#         print(batch_pk.shape)
        mean[i]=torch.mean(batch_pk,axis=0)
        sdev[i]=torch.std(batch_pk,axis=0)
        
    mean=torch.stack(mean)
    sdev=torch.stack(sdev)
    return mean,sdev


In [74]:
## Stored mean and std of spectrum for full input data once
mean_spec_data,sdev_spec_data=f_torch_image_spectrum(t_img[:1000],1)
hist_data=torch.histc(t_img[:1000],bins=50)


In [24]:
# real_cpu = data[0].to(device)
# f_torch_radial_profile(real_cpu[0,0,:,:])
# f_torch_compute_spectrum(real_cpu[0,0,:,:])
# f_torch_compute_batch_spectrum(real_cpu[:,0,:,:]).shape
# mean,sdev=f_torch_image_spectrum(real_cpu,1)

# mean2,sdev2=f_image_spectrum(real_cpu.cpu().numpy(),1)
# for i in range(len(mean.numpy())):
#     print (abs(mean.numpy()[i]-mean2[i]) <=1e-2)


In [25]:
def loss_spectrum(spec_mean,spec_mean_ref,spec_std,spec_std_ref,image_size):
    ''' Loss function for the spectrum : mean + variance '''
    
    # Log ( sum( batch value - expect value) ^ 2 ))
    
    idx=int(image_size/2) ### For the spectrum, use only N/2 indices for loss calc.
    
    spec_mean=torch.log(torch.mean(torch.pow(spec_mean[:,idx]-spec_mean_ref[:,idx],2)))
    spec_sdev=torch.log(torch.mean(torch.pow(spec_std[:,idx]-spec_std_ref[:,idx],2)))
    
    lambda1=1.0;lambda2=1.0;
    ans=lambda1*spec_mean+lambda2*spec_sdev
    return ans.item()


def loss_hist(data,hist_data):
    
    hist_sample=torch.histc(data,bins=50)
    ## A kind of normalization of histograms: divide by total sum
    hist_sample=hist_sample/torch.sum(hist_sample)
    hist_data=hist_data/torch.sum(hist_data)

    return torch.log(torch.mean(torch.pow(hist_sample-hist_data,2))).item()

# loss_spectrum(mean,mean_spec_data,sdev,sdev_spec_data,128)
# loss_hist(fake,hist_data)

In [ ]:
num_epochs=5

In [ ]:
### Create prefix for foldername 
now=datetime.now()
fldr_name=now.strftime('%Y%m%d_%H%M%S') ## time format
# print(fldr_name)
save_dir=op_loc+fldr_name

if not os.path.exists(save_dir):
    os.makedirs(save_dir+'/models')
    os.makedirs(save_dir+'/images')
    
    
keys=['Dreal','Dfake','Dfull','G','spec_chi','hist_chi']
size=len(dataset)/batch_size * num_epochs
metric_dict=dict(zip(keys,[np.empty((int(np.ceil(size))))*np.nan for i in range(len(keys))]))

In [ ]:
# Lists to keep track of progress

iters = 0
best_chi1,best_chi2=1e10,1e10
flip_prob=0.01


t0=time.time()
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for count, data in enumerate(dataloader, 0):
        tme1=time.time()
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        real_label = torch.full((b_size,), 1, device=device)
        fake_label = torch.full((b_size,), 0, device=device)
        
        ## Flip labels with probability flip_prob
#         for count,i in enumerate(real_label):
#             if torch.rand(1)[0] < flip_prob:
#                 real_label[count]=0
#                 fake_label[count]=1
        
        for idx in np.random.choice(np.arange(b_size),size=int(np.ceil(b_size*flip_prob))):
            real_label[idx]=0;
            fake_label[idx]=1;
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, real_label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 1, 1, nz, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, fake_label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        real_label = torch.full((b_size,), 1, device=device) ## No flipping for Generator labels
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, real_label)
        # Add spectral loss
        mean,sdev=f_torch_image_spectrum(fake,1)  ### compute spectral mean,std for fake images for batch
        spec_loss=loss_spectrum(mean,mean_spec_data,sdev,sdev_spec_data,image_size)
        
        # Combine losses
        errG+=spec_loss
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        
        tme2=time.time()
        # Output training stats
        if count % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, count, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2)),
            print("Time taken for step %s : %s"%(iters, tme2-tme1))

        
        # Histogram pixel intensity metric
        hist_metric=loss_hist(fake,hist_data.to(device))
        
        # Save metrics
        for key,val in zip(['Dreal','Dfake','Dfull','G','spec_chi','hist_chi'],[errD_real.item(),errD_fake.item(),errD.item(),errG.item(),spec_loss,hist_metric]):
            metric_dict[key][iters]=val
        
        ### Checkpoint the best model
        checkpoint=True
        if checkpoint and epoch > 3:
            # Choose best models by metric
            if hist_metric< best_chi1:
                torch.save({'epoch':epoch,'iters':iters,'G_state':netG.state_dict(),'D_state':netD.state_dict(),
               'optimizerG_state_dict':optimizerG.state_dict(),'optimizerD_state_dict':optimizerD.state_dict()}
              , save_dir+'/models/checkpoint_best_hist.tar')
                best_chi1=hist_metric
            
            if spec_loss< best_chi2:
                torch.save({'epoch':epoch,'iters':iters,'G_state':netG.state_dict(),'D_state':netD.state_dict(),
               'optimizerG_state_dict':optimizerG.state_dict(),'optimizerD_state_dict':optimizerD.state_dict()}
              , save_dir+'/models/checkpoint_best_spec.tar')
                best_chi2=spec_loss
        
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 50 == 0) or ((epoch == num_epochs-1) and (count == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
                img_arr=np.array(fake[:,0,:,:])
                fname='gen_img_epoch-%s_step-%s'%(epoch,iters)
                np.save(save_dir+'/images/'+fname,img_arr)

        iters += 1

tf=time.time()
print("Total time",tf-t0)

### Save Losses to files
with open (save_dir+'/metrics.pickle', 'wb') as f:
    pickle.dump(metric_dict,f)


In [ ]:
# ! jupyter nbconvert --to script cosmogan_test.ipynb


In [ ]:
print(metric_dict.keys())
metric_dict['spec_chi']
# metric_dict['hist_chi']

In [56]:
### Test the loss functions on results and keras results

In [86]:
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
img_1=np.expand_dims(np.load(f1)[:500],axis=1)

f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/gen_img_epoch-2_step-3150.npy'
f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/best_-hist_gen_img_epoch-4_step-7002.npy'
# f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/best_-spec_gen_img_epoch-5_step-8012.npy'
img_2=np.expand_dims(np.load(f2)[:500],axis=1)



In [87]:
loss_hist(torch.from_numpy(img_1),hist_data),loss_hist(torch.from_numpy(img_2),hist_data)

UnboundLocalError: local variable 'hist_data' referenced before assignment

In [88]:
mean1,sdev1=f_torch_image_spectrum(torch.from_numpy(img_1),1)  ### compute spectral mean,std for fake images for batch
mean2,sdev2=f_torch_image_spectrum(torch.from_numpy(img_2),1)  ### compute spectral mean,std for fake images for batch
loss_spectrum(mean1,mean_spec_data,sdev1,sdev_spec_data,128),loss_spectrum(mean2,mean_spec_data,sdev2,sdev_spec_data,128)

(0.07428026933517742, 0.096091710887477)

In [77]:
hist_data/torch.sum(hist_data)

tensor([4.3180e-02, 1.4124e-01, 1.5408e-01, 1.1212e-01, 7.6473e-02, 7.2860e-02,
        7.0680e-02, 5.5724e-02, 4.0763e-02, 3.2159e-02, 2.7141e-02, 2.2514e-02,
        1.8430e-02, 1.5644e-02, 1.3373e-02, 1.1398e-02, 9.9565e-03, 8.6518e-03,
        7.6531e-03, 6.7343e-03, 5.9770e-03, 5.3076e-03, 4.7750e-03, 4.2794e-03,
        3.8366e-03, 3.4656e-03, 3.1497e-03, 2.8642e-03, 2.6256e-03, 2.3782e-03,
        2.1692e-03, 2.0018e-03, 1.8183e-03, 1.6394e-03, 1.5142e-03, 1.3965e-03,
        1.2879e-03, 1.1665e-03, 1.0861e-03, 9.8065e-04, 8.9984e-04, 8.2343e-04,
        7.4542e-04, 6.7346e-04, 6.0468e-04, 5.4163e-04, 4.6198e-04, 3.8287e-04,
        2.7393e-04, 9.9731e-05])

In [71]:
# loss_hist(hist_data,hist_data)
loss_spectrum(mean1,mean_spec_data,sdev1,sdev_spec_data,128)

0.07428026933517742

In [75]:
hist_data

tensor([7.0747e+05, 2.3141e+06, 2.5244e+06, 1.8370e+06, 1.2529e+06, 1.1937e+06,
        1.1580e+06, 9.1298e+05, 6.6787e+05, 5.2690e+05, 4.4467e+05, 3.6887e+05,
        3.0196e+05, 2.5632e+05, 2.1910e+05, 1.8674e+05, 1.6313e+05, 1.4175e+05,
        1.2539e+05, 1.1033e+05, 9.7927e+04, 8.6959e+04, 7.8233e+04, 7.0113e+04,
        6.2859e+04, 5.6780e+04, 5.1605e+04, 4.6927e+04, 4.3018e+04, 3.8964e+04,
        3.5540e+04, 3.2798e+04, 2.9791e+04, 2.6860e+04, 2.4808e+04, 2.2881e+04,
        2.1101e+04, 1.9112e+04, 1.7794e+04, 1.6067e+04, 1.4743e+04, 1.3491e+04,
        1.2213e+04, 1.1034e+04, 9.9070e+03, 8.8740e+03, 7.5690e+03, 6.2730e+03,
        4.4880e+03, 1.6340e+03])

In [69]:
def loss_spectrum(spec_mean,spec_mean_ref,spec_std,spec_std_ref,image_size):
    ''' Loss function for the spectrum : mean + variance '''
    
    # Log ( sum( batch value - expect value) ^ 2 ))
    
    idx=int(image_size/2) ### For the spectrum, use only N/2 indices for loss calc.
    
    spec_mean=torch.mean(torch.pow(spec_mean[:,idx]-spec_mean_ref[:,idx],2))
    spec_sdev=torch.mean(torch.pow(spec_std[:,idx]-spec_std_ref[:,idx],2))
    
    
    lambda1=1.0;lambda2=1.0;
    ans=lambda1*spec_mean+lambda2*spec_sdev
    return ans.item()


def loss_hist(data,hist_da):
    
    hist_sample=torch.histc(data,bins=50)
    ## A kind of normalization of histograms: divide by total sum
    hist_sample=hist_sample/torch.sum(hist_sample)
    hist_data=hist_data/torch.sum(hist_data)

    return torch.log(torch.mean(torch.pow(hist_sample-hist_data,2))).item()
#     return torch.mean(torch.pow(hist_sample-hist_data,2)).item()